In [22]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import numbers
import json
import copy

In [23]:
def standard_request(url):
    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }
    with requests.Session() as s:
        r = s.get(url, headers=req_headers)
    sresponse = r.content
    soup1 = BeautifulSoup(sresponse, 'html.parser')
    G = json.loads(json.loads(soup1.find_all(string=re.compile('apiCache'))[0])['apiCache']) # keys: 'VariantQuery{"zpid":48875685,"altId":null}' 'ForSaleDoubleScrollFullRenderQuery{"zpid":48875685,"contactFormRenderParameter":{"zpid":48875685,"platform":"desktop","isDoubleScroll":true}}'
    return G

def api_add(G, key, houses, key2):
    if 'lotSize' in G['VariantQuery{"zpid":'+zpid+',"altId":null}']['property'].keys():
        houses[key]['Lot Area'] = G['VariantQuery{"zpid":'+zpid+',"altId":null}']['property']['lotSize']
    else:
        houses[key]['Lot Area'] = None
    if 'yearBuilt' in G['VariantQuery{"zpid":'+zpid+',"altId":null}']['property'].keys():
        houses[key]['Year'] = G['VariantQuery{"zpid":'+zpid+',"altId":null}']['property']['yearBuilt']
    else:
        houses[key]['Year'] = None
    if 'ForSaleDoubleScrollFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}' in G.keys():
        if 'taxAnnualAmount' in G['ForSaleDoubleScrollFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['resoFacts'].keys():
            houses[key]['Annual Tax'] = G['ForSaleDoubleScrollFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['resoFacts']['taxAnnualAmount']
        else:
            houses[key]['Annual Tax'] = None
        if 'monthlyHoaFee' in G['ForSaleDoubleScrollFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property'].keys():
            houses[key]['Monthly HOA'] = G['ForSaleDoubleScrollFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['monthlyHoaFee']
        else:
            houses[key]['Monthly HOA'] = None
    elif 'ForSaleShopperPlatformFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}' in G.keys():
        if 'taxAnnualAmount' in G['ForSaleShopperPlatformFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['resoFacts'].keys():
            houses[key]['Annual Tax'] = G['ForSaleShopperPlatformFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['resoFacts']['taxAnnualAmount']
        else:
            houses[key]['Annual Tax'] = None
        if 'monthlyHoaFee' in G['ForSaleShopperPlatformFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property'].keys():
            houses[key]['Monthly HOA'] = G['ForSaleShopperPlatformFullRenderQuery{"zpid":'+zpid+',"contactFormRenderParameter":{"zpid":'+zpid+',"platform":"desktop","isDoubleScroll":true}}']['property']['monthlyHoaFee']
        else:
            houses[key]['Monthly HOA'] = None

In [25]:
# Conduct a search on Zillow by inserting the postal code.
houses = {}
pages = 9
for dummy12 in range(1, pages+1):
    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }
    with requests.Session() as s:
        city = 'nampa' #*****change this city to what you want*****
        state = 'ID'
        if dummy12 > 1:
            url = 'https://www.zillow.com/'+city+'-'+state+'/'+str(dummy12)+'_p/'
            print(url)
        else:
            url = 'https://www.zillow.com/homes/for_sale/'+city
        r = s.get(url, headers=req_headers)
     # https://www.zillow.com/homedetails/
    sresponse = r.content
    soup = BeautifulSoup(r.content, 'html.parser')
    url_json_str = soup.find_all(string = re.compile('"detailUrl":"https://www.zillow.com/homedetails'))
    detail_urls = json.loads(str(url_json_str[0])[4:-3])
    for link in detail_urls['cat1']['searchResults']['listResults']:
        temp = link['hdpData']['homeInfo']
        temp2 = link['detailUrl']
        if 'streetAddress' in temp.keys() and 'city' in temp.keys() and 'state' in temp.keys() and 'zipcode' in temp.keys():
            houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']] = {}
            if 'homeType' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Type'] = temp['homeType']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Type'] = None
            if 'price' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Flat Price'] = temp['price']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Flat Price'] = None
            if 'zestimate' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Zestimate'] = temp['zestimate']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['Zestimate'] = None
            if 'rentZestimate' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['rentZestimate'] = temp['rentZestimate']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['rentZestimate'] = None
            if 'taxAssessedValue' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['taxValue'] = temp['taxAssessedValue']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['taxValue'] = None
            if 'bathrooms' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['bathrooms'] = temp['bathrooms']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['bathrooms'] = None
            if 'bedrooms' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['bedrooms'] = temp['bedrooms']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['bedrooms'] = None
            if temp2:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['url'] = temp2
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['url'] = None
            if 'livingArea' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['livingArea'] = temp['livingArea']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['livingArea'] = None
            if 'zpid' in temp.keys():
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['zpid'] = temp['zpid']
            else:
                houses[temp['streetAddress'] + " " + temp['city'] + " " + temp['state'] + " " + temp['zipcode']]['zpid'] = None
for dic in houses.items():
    G = standard_request(dic[1]['url'])
    zpid = str(dic[1]['zpid'])
    key2 = 'VariantQuery{"zpid":' + str(dic[1]['zpid']) + ',"altId":null}'
    key = G[key2]['property']['streetAddress'] + " " + G[key2]['property']['city'] + " " + G[key2]['property']['state'] + " " + G[key2]['property']['zipcode']
    api_add(G, key, houses, zpid)

https://www.zillow.com/nampa-ID/2_p/
https://www.zillow.com/nampa-ID/3_p/
https://www.zillow.com/nampa-ID/4_p/
https://www.zillow.com/nampa-ID/5_p/
https://www.zillow.com/nampa-ID/6_p/
https://www.zillow.com/nampa-ID/7_p/
https://www.zillow.com/nampa-ID/8_p/
https://www.zillow.com/nampa-ID/9_p/


In [26]:
print(houses)

{'208 S Diamond St Nampa ID 83686': {'Type': 'SINGLE_FAMILY', 'Flat Price': 289900.0, 'Zestimate': 285500, 'rentZestimate': 1194, 'taxValue': 172500.0, 'bathrooms': 2.0, 'bedrooms': 2.0, 'url': 'https://www.zillow.com/homedetails/208-S-Diamond-St-Nampa-ID-83686/72336741_zpid/', 'livingArea': 1109.0, 'zpid': 72336741, 'Lot Area': 6969.6, 'Year': 1920, 'Annual Tax': 1016, 'Monthly HOA': None}, '1208 S Florida Pl Nampa ID 83686': {'Type': 'SINGLE_FAMILY', 'Flat Price': 375000.0, 'Zestimate': 376800, 'rentZestimate': 1798, 'taxValue': 270400.0, 'bathrooms': 2.0, 'bedrooms': 3.0, 'url': 'https://www.zillow.com/homedetails/1208-S-Florida-Pl-Nampa-ID-83686/72321877_zpid/', 'livingArea': 1379.0, 'zpid': 72321877, 'Lot Area': 7623, 'Year': 1999, 'Annual Tax': 2881, 'Monthly HOA': 18}, '12635 Shadow Peak Caldwell ID 83607': {'Type': 'SINGLE_FAMILY', 'Flat Price': 436990.0, 'Zestimate': None, 'rentZestimate': None, 'taxValue': None, 'bathrooms': 3.0, 'bedrooms': 4.0, 'url': 'https://www.zillow.co

In [27]:
for address in houses.keys():
    print(houses[address].keys())
    if 'Monthly HOA' not in houses[address].keys():
        houses[address]['Monthly HOA'] = None
    if 'Annual Tax' not in houses[address].keys():
        houses[address]['Annual Tax'] = None
    if 'Year' not in houses[address].keys():
        houses[address]['Year'] = None
    if 'Lot Area' not in houses[address].keys():
        houses[address]['Lot Area'] = None

dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url', 'livingArea', 'zpid', 'Lot Area', 'Year', 'Annual Tax', 'Monthly HOA'])
dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url', 'livingArea', 'zpid', 'Lot Area', 'Year', 'Annual Tax', 'Monthly HOA'])
dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url', 'livingArea', 'zpid', 'Lot Area', 'Year'])
dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url', 'livingArea', 'zpid', 'Lot Area', 'Year', 'Annual Tax', 'Monthly HOA'])
dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url', 'livingArea', 'zpid', 'Lot Area', 'Year', 'Annual Tax', 'Monthly HOA'])
dict_keys(['Type', 'Flat Price', 'Zestimate', 'rentZestimate', 'taxValue', 'bathrooms', 'bedrooms', 'url', 'livingArea', 'zpid', 'Lot 

In [28]:
houses2 = {}
order = list(houses.keys())
houses2['address'] = []
for key in houses[order[0]].keys():
    houses2[key] = []


for house in order:
    for key in houses[house].keys():
        houses2[key].append(houses[house][key])
    houses2['address'].append(house)

In [29]:
for j in houses2.keys():
    print(len(houses2[j]), j)

350 address
350 Type
350 Flat Price
350 Zestimate
350 rentZestimate
350 taxValue
350 bathrooms
350 bedrooms
350 url
350 livingArea
350 zpid
350 Lot Area
350 Year
350 Annual Tax
350 Monthly HOA


In [30]:
houses3 = pd.DataFrame.from_dict(houses2)

In [31]:
houses3.to_excel("Nampa_ID.xlsx")